# Swap HDX
Agent $\alpha$ swaps asset $i$ with the protocol for HDX.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import inspect
from model.amm.omnipool_amm import swap_lhdx, swap_lhdx_fee, swap_hdx_delta_Qi, swap_hdx_delta_Ri

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta R_i \leq r_i^\alpha
$$
#### Case 2: HDX sold
$$
\Delta Q_i \leq q^\alpha
$$

## Updating AMM state

### Case 1: Asset $i$ sold

$$
\Delta Q_i = Q_i\frac{- \Delta R_i}{R_i + \Delta R_i}
$$

In [2]:
print(inspect.getsource(swap_hdx_delta_Qi))

def swap_hdx_delta_Qi(old_state: dict, delta_Ri: float, i: int) -> float:
    return old_state['Q'][i] * (- delta_Ri / (old_state['R'][i] + delta_Ri))



### Case 2: LHDX sold

$$
\Delta R_i = R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}
$$

In [3]:
print(inspect.getsource(swap_hdx_delta_Ri))

def swap_hdx_delta_Ri(old_state: dict, delta_Qi: float, i: int) -> float:
    return old_state['R'][i] * (- delta_Qi / (old_state['Q'][i] + delta_Qi))



## Updating agent state

### Case 1: LHDX bought from pool
In this case, the LHDX fee is sent to treasury.
$$
\Delta q^\alpha = - \Delta Q_i\left(1 - f_Q\right)\\
\Delta r_i^\alpha = - \Delta R_i\\
\Delta D = - \Delta Q_i f_Q
$$

### Case 2: HDX sold to pool
In this case, we distribute the fee (collected in $i$) to the LPs of asset $i$. This is accomplished by adding the fee amount of asset $i$ back to the pool, and minting LHDX such that the price does not change during this liquidity add. Note that this differs from a normal liquidity add in that no new shares are minted.
$$
\Delta q^\alpha = - \Delta Q_i\\
\Delta r_i^\alpha = - \Delta R_i\left(1 - f_R\right)\\
R_i^+ = R_i + \Delta R_i (1 - f_R)\\
Q_i^+ = R_i^+ \frac{Q_i + \Delta Q_i}{R_i + \Delta R_i}
$$

In [4]:
print(inspect.getsource(swap_lhdx))
print(inspect.getsource(swap_lhdx_fee))

def swap_lhdx(
        old_state: dict,
        old_agents: dict,
        trader_id: string,
        delta_R: float,
        delta_Q: float,
        i: int
) -> tuple:
    """Compute new state after HDX swap"""

    new_state = copy.deepcopy(old_state)
    new_agents = copy.deepcopy(old_agents)

    if delta_Q == 0 and delta_R != 0:
        delta_Q = swap_hdx_delta_Qi(old_state, delta_R, i)
    elif delta_R == 0 and delta_Q != 0:
        delta_R = swap_hdx_delta_Ri(old_state, delta_Q, i)
    else:
        return new_state, new_agents

    # Token amounts update
    if delta_Q < 0:
        new_state['R'][i] += delta_R
        new_state['Q'][i] += delta_Q
        new_agents[trader_id]['r'][i] -= delta_R
        new_agents[trader_id]['q'] -= delta_Q

    else:
        new_state['R'][i] += delta_R
        new_state['Q'][i] = (old_state['Q'][i] + delta_Q) / (old_state['R'][i] + delta_R) * new_state['R'][i]
        new_agents[trader_id]['r'][i] -= delta_R
        new_agents[trader_id]['q'] -